### Exploration of a pdf called cross culture which is a catalog of the allard pierson museum

In [6]:
# importing required modules 
import PyPDF2 
    
# creating a pdf file object 
pdfFileObj = open('data/CrossRoads_ENGELS_LR_compleet.pdf', 'rb') 
    
# creating a pdf reader object 
pdfReader = PyPDF2.PdfFileReader(pdfFileObj) 
    
# printing number of pages in pdf file 
print(pdfReader.numPages) 
    
# creating a page object 
pageObj = pdfReader.getPage(0) 
    
# extracting text from page 
print(pageObj.extractText()) 
    
# closing the pdf file object 
pdfFileObj.close() 

212
www.allardpiersonmuseum.nlCROSSROADS
A runic graffito in the Hagia Sophia, a gilt Byzantine 
helmet in the grave of a Frankish nobleman, a trea­
sure hidden from the Vikings in the Low Countries  containing an Arab dirham: these are just a few exam ­
ples of the telling early ­medieval finds in this book. 
Late Antiquity and the Early Middle Ages are often 
viewed as a time of decline, chaos, invasions and  war. But there is another side to this period as well. 
There was a rich diversity of cultures in Europe —  
from Longo  bards and Merovingians to Byzantines 
and Avars — and a lively exchange of goods and 
ideas, sometimes over great distances. The Vikings  
set up a trade network that reached to Baghdad;  
the Silk Road brought commodities to Europe, but also diplomatic missions, knowledge and ideas.  
This is illustrated by the interludes in this book, the 
stories of ten travellers: pilgrims, scholars, diplomats, 
and an elephant.
Despite the numerous conflicts, the period f

extract text and obtain csv file

In [12]:
import csv

fields = ['text']
filename = 'crossroads.csv'

with open(filename, 'w') as csvfile:
    # creating a csv writer object 
    csvwriter = csv.writer(csvfile) 
        
    # writing the fields 
    csvwriter.writerow(fields) 

    pdfFileObj = open('data/CrossRoads_ENGELS_LR_compleet.pdf', 'rb') 
    
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj) 

    for pagei in range(pdfReader.numPages):

        # creating a page object 
        pageObj = pdfReader.getPage(pagei) 
            
        csvwriter.writerow([pageObj.extractText()])
            
    # closing the pdf file object 
    pdfFileObj.close() 


In [13]:
import pandas as pd

filename='crossroads.csv'

df = pd.read_csv(filename)


In [14]:
df

,text
0,www.allardpiersonmuseum.nlCROSSROADS\nA runic ...
1,www.allardpiersonmuseum.nlCROSSROADS\nA runic ...
2,"Travelling through the Middle Ages, \nAD 300–1..."
3,This joint publication by the Allard Pierson M...
4,EDITORIAL BOARD\nMaria Bormpoudaki\nMarieke va...
...,...
207,206 CROSSROADS INFORMATIONKianoosh Rezania is ...
208,207 AUTHORS’ BIOGRAPHIESILLUSTRATION \nCREDITS...
209,COLOPHONLandesamt für Kultur und \nDenkmalpfl...
210,www.allardpiersonmuseum.nlCROSSROADS\nA runic ...


## image matching

In [35]:
import cv2 
import matplotlib.pyplot as plt

# read images
img1 = cv2.imread('images/img-22.png')  
img2 = cv2.imread('data/example.jpg') 

img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

#sift
sift = cv2.xfeatures2d.SIFT_create()

keypoints_1, descriptors_1 = sift.detectAndCompute(img1,None)
keypoints_2, descriptors_2 = sift.detectAndCompute(img2,None)

#feature matching
bf = cv2.BFMatcher(cv2.NORM_L1, crossCheck=True)

matches = bf.match(descriptors_1,descriptors_2)
matches = sorted(matches, key = lambda x:x.distance)

img3 = cv2.drawMatches(img1, keypoints_1, img2, keypoints_2, matches[:50], img2, flags=2)
plt.imshow(img3),plt.show()

AttributeError: module 'cv2' has no attribute 'xfeatures2d'

image comparison

In [36]:
import cv2
  
     
# test image
image = cv2.imread('data/example.jpg')  
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
histogram = cv2.calcHist([gray_image], [0],
                         None, [256], [0, 256])
  
# data1 image
image = cv2.imread('images/img-22.png')
gray_image1 = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
histogram1 = cv2.calcHist([gray_image1], [0],
                          None, [256], [0, 256])
  
# data2 image
image = cv2.imread('images/img-26.png')
gray_image2 = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
histogram2 = cv2.calcHist([gray_image2], [0],
                          None, [256], [0, 256])
  
  
c1, c2 = 0, 0
  
# Euclidean Distance between data1 and test
i = 0
while i<len(histogram) and i<len(histogram1):
    c1+=(histogram[i]-histogram1[i])**2
    i+= 1
c1 = c1**(1 / 2)
  
 
# Euclidean Distance between data2 and test
i = 0
while i<len(histogram) and i<len(histogram2):
    c2+=(histogram[i]-histogram2[i])**2
    i+= 1
c2 = c2**(1 / 2)
  
if(c1<c2):
    print("data1.jpg is more similar to test.jpg as compare to data2.jpg")
else:
    print("data2.jpg is more similar to test.jpg as compare to data1.jpg")

data1.jpg is more similar to test.jpg as compare to data2.jpg


## k-means image clustering
code from: https://towardsdatascience.com/how-to-cluster-images-based-on-visual-similarity-cd6e7209fe34

In [1]:
# for loading/processing the images  
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array
from keras.applications.vgg16 import preprocess_input 

# models 
from keras.applications.vgg16 import VGG16 
from keras.models import Model

# clustering and dimension reduction
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.decomposition import PCA

# for everything else
import os
import numpy as np
import matplotlib.pyplot as plt
from random import randint
import pandas as pd
import pickle

path = r"/mnt/c/Users/dariu/Documents/THESIS/CUE/images"
# change the working directory to the path where the images are located
os.chdir(path)

# this list holds all the image filename
flowers = []

# creates a ScandirIterator aliased as files
with os.scandir(path) as files:
  # loops through each file in the directory
    for file in files:
        if file.name.endswith('.png'):
          # adds only the image files to the flowers list
            flowers.append(file.name)
            
            
            
model = VGG16()
model = Model(inputs = model.inputs, outputs = model.layers[-2].output)

def extract_features(file, model):
    # load the image as a 224x224 array
    img = load_img(file, target_size=(224,224))
    # convert from 'PIL.Image.Image' to numpy array
    img = np.array(img) 
    # reshape the data for the model reshape(num_of_samples, dim 1, dim 2, channels)
    reshaped_img = img.reshape(1,224,224,3) 
    # prepare image for model
    imgx = preprocess_input(reshaped_img)
    # get the feature vector
    features = model.predict(imgx, use_multiprocessing=True)
    return features
   
data = {}
p = r"data/clusters"

# lop through each image in the dataset
for flower in flowers:
    # try to extract the features and update the dictionary
    try:
        feat = extract_features(flower,model)
        data[flower] = feat
    # if something fails, save the extracted features as a pickle file (optional)
    except:
        with open(p,'wb') as file:
            pickle.dump(data,file)
          
print('features extracted')

# get a list of the filenames
filenames = np.array(list(data.keys()))

# get a list of just the features
feat = np.array(list(data.values()))

# reshape so that there are 210 samples of 4096 vectors
feat = feat.reshape(-1,4096)

# get the unique labels (from the flower_labels.csv)

# no fixed number of cluster available. 

# df = pd.read_csv('flower_labels.csv')
# label = df['label'].tolist()
# unique_labels = list(set(label))

print('applying pca')

# reduce the amount of dimensions in the feature vector
pca = PCA(n_components=15, random_state=22)
pca.fit(feat)
x = pca.transform(feat)

print('pca done')
print('starting clustering')
# cluster feature vectors
kmeans = KMeans(n_clusters=4, random_state=22, n_init=3, max_iter=100)
kmeans.fit(x)
print('clustering done')

# holds the cluster id and the images { id: [images] }
groups = {}
for file, cluster in zip(filenames,kmeans.labels_):
    if cluster not in groups.keys():
        groups[cluster] = []
        groups[cluster].append(file)
    else:
        groups[cluster].append(file)

# function that lets you view a cluster (based on identifier)        
def view_cluster(cluster):
    plt.figure(figsize = (25,25))
    # gets the list of filenames for a cluster
    files = groups[cluster]
    # only allow up to 30 images to be shown at a time
    if len(files) > 30:
        print(f"Clipping cluster size from {len(files)} to 30")
        files = files[:29]
    # plot each image in the cluster
    for index, file in enumerate(files):
        plt.subplot(10,10,index+1)
        img = load_img(file)
        img = np.array(img)
        plt.imshow(img)
        plt.axis('off')
        
   
# this is just incase you want to see which value for k might be the best 
sse = []
list_k = list(range(3, 50))

for k in list_k:
    km = KMeans(n_clusters=k, random_state=22)
    km.fit(x)
    
    sse.append(km.inertia_)

# Plot sse against k
plt.figure(figsize=(6, 6))
plt.plot(list_k, sse)
plt.xlabel(r'Number of clusters *k*')
plt.ylabel('Sum of squared distance')

2022-11-21 15:35:34.400261: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-21 15:35:37.104992: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-21 15:35:37.107213: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


1/1 [==============================] - 0s 212ms/step
features extracted
applying pca
pca done
starting clustering
